# Draft / Ideas: Multilingual Evaluation

## Overview
This notebook evaluates the fine-tuned XLM-RoBERTa model on custom text (English).

In [ ]:
# Import required libraries
import json
from pathlib import Path
from typing import List, Dict

# Hugging Face libraries
from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    pipeline
)
import torch

import warnings
warnings.filterwarnings('ignore')

print("✓ Libraries imported")

In [2]:
# Configure paths
MODEL_PATH = Path("../models/litbank-xlm-roberta")
PROCESSED_DATA_PATH = Path("../data/processed")
RESULTS_PATH = Path("../results")
RESULTS_PATH.mkdir(parents=True, exist_ok=True)

# Check if model exists
if not MODEL_PATH.exists():
    print(f"⚠️  Model not found at {MODEL_PATH}")
    print("Please run Notebook 2 (Model Training) first to train the model.")
else:
    print(f"✓ Found trained model at: {MODEL_PATH.absolute()}")

✓ Found trained model at: c:\Users\niw\Documents\CAS NLP Mod4\Mod4_Project\module4-transformers\notebooks\..\models\litbank-xlm-roberta


## 1. Load Fine-tuned Model

Load the XLM-RoBERTa model trained on English LitBank data.

In [3]:
# Load model and tokenizer
print("Loading fine-tuned model...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForTokenClassification.from_pretrained(MODEL_PATH)

# Load label mapping
with open(PROCESSED_DATA_PATH / "label_mapping.json", 'r') as f:
    label_mapping = json.load(f)

label2id = label_mapping["label2id"]
id2label = {int(k): v for k, v in label_mapping["id2label"].items()}

print(f"\n✓ Model loaded successfully")
print(f"  Model: XLM-RoBERTa fine-tuned on LitBank")
print(f"  Parameters: {model.num_parameters():,}")
print(f"  Entity types: {len([l for l in label2id if l.startswith('B-')])}")
print(f"\nSupported entity types:")
entity_types = sorted(set([l[2:] for l in label2id.keys() if l.startswith('B-')]))
for entity_type in entity_types:
    print(f"  - {entity_type}")

Loading fine-tuned model...


The tokenizer you are loading from '..\models\litbank-xlm-roberta' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.



✓ Model loaded successfully
  Model: XLM-RoBERTa fine-tuned on LitBank
  Parameters: 277,463,053
  Entity types: 7

Supported entity types:
  - FAC
  - GPE
  - LOC
  - ORG
  - PER
  - TIME
  - VEH


## 2. Create NER Pipeline

- Simplifies inference (handles tokenization, prediction, decoding)
- Aggregates subword predictions into word-level entities
- Provides confidence scores

Example test passage from Charles Dickens' *Christmas Carol*

In [4]:
# Create NER pipeline
ner_pipeline = pipeline(
    "token-classification",
    model=model,
    tokenizer=tokenizer,
    aggregation_strategy="simple",  # Aggregate subwords into entities
    device=0 if torch.cuda.is_available() else -1  # Use GPU if available
)

print("✓ NER pipeline created")
print(f"  Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")

# Test on example sentence
test_sentence = "Although they had but that moment left the school behind them, they were now in the busy thoroughfares of a city, where shadowy passengers passed and re-passed; where shadowy carts and coaches battled for the way, and all the strife and tumult of a real city were. It was made plain enough, by the dressing of the shops, that here, too, it was Christmas-time again; but it was evening, and the streets were lighted up. The Ghost stopped at a certain warehouse door, and asked Scrooge if he knew it."
print(f"\nTest prediction:")
print(f"  Input: {test_sentence}")
predictions = ner_pipeline(test_sentence)
print(f"\n  Detected entities:")
for entity in predictions:
    print(f"    - {entity['word']:20s} → {entity['entity_group']:6s} (confidence: {entity['score']:.3f})")

Device set to use cpu


✓ NER pipeline created
  Device: CPU

Test prediction:
  Input: Although they had but that moment left the school behind them, they were now in the busy thoroughfares of a city, where shadowy passengers passed and re-passed; where shadowy carts and coaches battled for the way, and all the strife and tumult of a real city were. It was made plain enough, by the dressing of the shops, that here, too, it was Christmas-time again; but it was evening, and the streets were lighted up. The Ghost stopped at a certain warehouse door, and asked Scrooge if he knew it.

  Detected entities:
    - the school           → FAC    (confidence: 0.812)
    - the busy thoroughfares of → FAC    (confidence: 0.907)
    - a city               → GPE    (confidence: 0.868)
    - ,                    → FAC    (confidence: 0.543)
    - shadowy passengers   → PER    (confidence: 0.962)
    - shadowy carts        → VEH    (confidence: 0.508)
    - coaches              → VEH    (confidence: 0.497)
    - a real city 

In [6]:
# Color-coded NER visualization
from IPython.display import HTML, display

def display_ner_results(text: str, entities: List[Dict], show_confidence: bool = True):
    """
    Display NER results with color-coded entity tags.
    
    Args:
        text: Input text
        entities: List of entity predictions from the pipeline
        show_confidence: Whether to show confidence scores
    """
    # Define colors for each entity type
    entity_colors = {
        'PER': '#ffa07a',  # Light salmon
        'ORG': '#87ceeb',  # Sky blue
        'LOC': '#98fb98',  # Pale green
        'GPE': '#dda0dd',  # Plum
        'FAC': '#f0e68c',  # Khaki
        'VEH': '#ffb6c1',  # Light pink
    }
    
    # Sort entities by their start position
    sorted_entities = sorted(entities, key=lambda x: x['start'])
    
    # Build HTML with highlighted entities
    html_parts = []
    last_end = 0
    
    for entity in sorted_entities:
        start = entity['start']
        end = entity['end']
        entity_text = entity['word']
        entity_type = entity['entity_group']
        confidence = entity['score']
        
        # Add text before entity
        if start > last_end:
            html_parts.append(text[last_end:start])
        
        # Add highlighted entity
        color = entity_colors.get(entity_type, '#d3d3d3')
        tooltip = f"{entity_type}"
        if show_confidence:
            tooltip += f" ({confidence:.2f})"
        
        html_parts.append(
            f'<mark style="background-color: {color}; padding: 2px 4px; '
            f'margin: 0 2px; border-radius: 3px; font-weight: bold;" '
            f'title="{tooltip}">'
            f'{entity_text} '
            f'<sup style="font-size: 0.7em; color: #555;">[{entity_type}]</sup>'
            f'</mark>'
        )
        
        last_end = end
    
    # Add remaining text
    if last_end < len(text):
        html_parts.append(text[last_end:])
    
    # Create legend
    legend_html = '<div style="margin-top: 20px; padding: 10px; background-color: #f5f5f5; border-radius: 5px;">'
    legend_html += '<strong>Entity Types:</strong><br>'
    for entity_type, color in entity_colors.items():
        legend_html += f'<span style="background-color: {color}; padding: 2px 8px; margin: 2px; border-radius: 3px; display: inline-block;">{entity_type}</span> '
    legend_html += '</div>'
    
    # Combine everything
    full_html = f'<div style="line-height: 2.0; font-size: 14px;">{"".join(html_parts)}</div>{legend_html}'
    
    display(HTML(full_html))

# Display the test sentence with color-coded entities
print("NER Results:")
display_ner_results(test_sentence, predictions, show_confidence=True)

NER Results:
